In [1]:
#dependency package for this notebook
# !pip install h5py
# !pip install torchinfo

In [10]:
import os
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, TensorDataset, Dataset
from torchvision.utils import make_grid
import matplotlib
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms, models
%matplotlib inline
import h5py
from torchinfo import summary
import tarfile
print(torch.__version__)

2.0.1


In [4]:
proton_url = "https://cernbox.cern.ch/remote.php/dav/public-files/AtBT8y4MiQYFcgc/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5"
download_url(proton_url, "./data/proton_data")

100%|██████████████████████████████████████████████████████████████████████████| 119703858/119703858 [01:38<00:00, 1220032.68it/s]


In [5]:
electron_url = "https://cernbox.cern.ch/remote.php/dav/public-files/FbXw3V4XNyYB3oA/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5"
download_url(electron_url, "./data/electron_data")

100%|██████████████████████████████████████████████████████████████████████████| 128927319/128927319 [01:40<00:00, 1284295.61it/s]


In [6]:
path_proton = "./data/proton_data/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5"
path_electron = "./data/electron_data/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5"
f = h5py.File(path_proton)
dset = f['X']
dset.shape
#f.keys()
#dset.attrs.keys()
#dset.attrs.values()
#dset.attrs.items()

(249000, 32, 32, 2)

In [7]:
class CustomDataset(Dataset):
    def __init__(self, path_electron):
        self.path = path_electron
        self.file = h5py.File(self.path)
        self.data = self.file['X']
        self.label = self.file['y']
        self.length = len(self.data)
        self.transform = transforms.ToTensor()

        
    def __len__(self):
        return self.length
        
    def __getitem__(self, index):
        image = self.data[index]
        image = self.transform(image)
        label = self.label[index]
        label = torch.tensor(label).long()
        return image, label
        
        

In [8]:
dataset = CustomDataset(path_electron)
train_dataloader = DataLoader(dataset, batch_size = 64, shuffle = True)
for images, labels in train_dataloader:
    print(f'{images[0].shape}')
    break

torch.Size([2, 32, 32])


In [9]:
model = models.resnet18(pretrained = True)
for param in model.parameters():
    param.requires_grad = False

model.fc.requires_grad = True

new_layer = nn.Sequential(
    nn.Linear(512, 256), 
    nn.ReLU(),
    nn.Dropout(p=0.5), 
    nn.Linear(256, 2),  
    nn.BatchNorm1d(2) 
)
model.fc = new_layer
summary(model = model, 
        input_size=(32, 3, 224, 224),
        col_names=["input_size", "output_size", "trainable"],
        #col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
       )

#loss_fn = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr = 0.001)

/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/bhushanbharat/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|████████████████████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [00:36<00:00, 1.28MB/s]


Layer (type (var_name))                  Input Shape          Output Shape         Trainable
ResNet (ResNet)                          [32, 3, 224, 224]    [32, 2]              Partial
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 64, 112, 112]   False
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]   [32, 64, 112, 112]   False
├─ReLU (relu)                            [32, 64, 112, 112]   [32, 64, 112, 112]   --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]   [32, 64, 56, 56]     --
├─Sequential (layer1)                    [32, 64, 56, 56]     [32, 64, 56, 56]     False
│    └─BasicBlock (0)                    [32, 64, 56, 56]     [32, 64, 56, 56]     False
│    │    └─Conv2d (conv1)               [32, 64, 56, 56]     [32, 64, 56, 56]     False
│    │    └─BatchNorm2d (bn1)            [32, 64, 56, 56]     [32, 64, 56, 56]     False
│    │    └─ReLU (relu)                  [32, 64, 56, 56]     [32, 64, 56, 56]     --
│    │    └─Conv2d (conv